## Summary of notebook:

This notebook shows the tuning process to obtain the optimal model architecture and hyperparameters for the Resnet-18 model using the **keras-tuner** package

Terminal's command: ```pip install keras-tuner```

The following shows some of the important details tuning process:
- Dataset used: Dataset augmented with Gaussian noise (augmented_features_10_ue1_v2_ds.npy\augmented_labels_10_ue1_v2_ds.npy)
- Tuner: Bayesian Optimiser
- Max trials: 50

Optimal Hyperparameters:
- fc_dropout: True
- pooling_dropout: False
- learning_rate: 0.0001
- l2_conv2d: 0.0
- l2_sepconv2d: 0.01
- batch_size: 16

In [17]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D, Add, Dense, \
BatchNormalization, ReLU, MaxPool2D, GlobalAvgPool2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model

In [18]:
# Configure amd test GPU
import tensorflow as tf
from tensorflow.python.client import device_lib

# Prevent automatic GPU memory pre-allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

print(tf.__version__)
# print(device_lib.list_local_devices())

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
2.9.1


In [3]:
# for kaggle
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/augmented-dataset-outdoor-v2/augmented_features_10_ue1_v2_ds.npy
/kaggle/input/augmented-dataset-outdoor-v2/augmented_labels_10_ue1_v2_ds.npy


In [19]:
import keras_tuner as kt

class HyperModel(kt.HyperModel):
    
    def build(self, hp):
        
        fc_dropout = hp.Boolean('fc_dropout', default = False)
        pooling_dropout = hp.Boolean('pooling_dropout', default = False)
        lr = hp.Choice('learning_rate', [0.01, 0.001, 0.0001])
        l2_conv2d = hp.Choice('l2_conv2d', [0.0, 0.01])
        l2_sepconv2d = hp.Choice('l2_sepconv2d', [0.0, 0.01])
        
        # Conv2D + Batch Normalization
        def conv_bn(x, filters, kernel_size, strides = 1):
            x = Conv2D(filters = filters, kernel_size = kernel_size,
                   strides = strides, padding = 'same', use_bias = False,
                      kernel_regularizer = keras.regularizers.L2(l2_conv2d))(x)
            x = BatchNormalization()(x)
            return x

        # Make a seperable convolutional block (Seperable Conv2D + Batch Normalization)
        def sep_bn(x, filters, kernel_size, strides = 1):
            x = SeparableConv2D(filters = filters, kernel_size = kernel_size, strides = strides,
                           padding = 'same', use_bias = False, kernel_regularizer = keras.regularizers.L2(l2_sepconv2d))(x)
            x = BatchNormalization()(x)
            return x
        
        # Construct the entry flow
        def entry_flow(x):

            # conv_bn(x, filters, kernel_size, strides)
            x = conv_bn(x, filters = 32, kernel_size = 3, strides = 2)
            x = ReLU()(x)

            x = conv_bn(x, filters = 64, kernel_size = 3)
            tensor = ReLU()(x)

            # Normal flow
            x = sep_bn(tensor, filters = 128, kernel_size = 3)
            x = ReLU()(x)
            x = sep_bn(x, filters = 128, kernel_size = 3)
            x = MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(x)
            
            if pooling_dropout:
                x = Dropout(rate = 0.2)(x)

            # Skip connection
            tensor = conv_bn(tensor, filters = 128, kernel_size = 1, strides = 2)
            x = Add()([x, tensor])
            # End of first skip connection

            # Skip connection
            tensor = conv_bn(tensor, filters = 256, kernel_size = 1, strides = 2)

            # Normal flow
            x = ReLU()(x)
            x = sep_bn(x, filters = 256, kernel_size = 3)
            x = ReLU()(x)
            x = sep_bn(x, filters = 256, kernel_size = 3)
            x = MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(x)
            
            if pooling_dropout:
                x = Dropout(rate = 0.2)(x)
                
            x = Add()([x, tensor])

            # End of second skip connection

            # Skip connection
            tensor = conv_bn(tensor, filters = 728, kernel_size = 1, strides = 2)

            # Normal flow
            x = ReLU()(x)
            x = sep_bn(x, filters = 728, kernel_size = 3)
            x = ReLU()(x)
            x = sep_bn(x, filters = 728, kernel_size = 3)
            x = MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(x)
            
            if pooling_dropout:
                x = Dropout(rate = 0.2)(x)
                
            x = Add()([x, tensor])

            # End of third skip connection

            return x
        
        # Construct the middle flow
        # Combine output from the entry flow and the convolution layers in the middle flow
        def middle_flow(tensor):

            for _ in range(8):

                x = ReLU()(tensor)
                x = sep_bn(x, filters = 728, kernel_size = 3)
                x = ReLU()(x)
                x = sep_bn(x, filters = 728, kernel_size = 3)
                x = ReLU()(x)
                x = sep_bn(x, filters = 728, kernel_size = 3)

                tensor = Add()([x, tensor])

            return x
        
        
        def exit_flow(tensor, num_classes = 1000):

            # Normal path
            x = ReLU()(tensor)
            x = sep_bn(x, filters = 728, kernel_size = 3)
            x = ReLU()(x)
            x = sep_bn(x, filters = 1024, kernel_size = 3)
            x = MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(x)
            
            if pooling_dropout:
                x = Dropout(rate = 0.2)(x)

            # Skip connection
            tensor = conv_bn(tensor, filters = 1024, kernel_size = 1, strides = 2)

            # Add outputs
            x = Add()([tensor, x])

            x = sep_bn(x, filters = 1536, kernel_size = 3)
            x = ReLU()(x)
            x = sep_bn(x, filters = 2048, kernel_size = 3)
            x = ReLU()(x)

            x = GlobalAvgPool2D()(x)
            
            if fc_dropout:
                x = Dropout(rate = 0.5)(x)
                
            x = Dense(units = num_classes, activation = 'softmax')(x)

            return x
        
        model_inputs = Input(shape = (193,16,1))
        model_outputs = exit_flow(middle_flow(entry_flow(model_inputs)), num_classes = 3876)
        xception_model = Model(model_inputs, model_outputs)

        # Check number of parameters
        # xception_model.summary()
        
        optimizer = tf.keras.optimizers.Adam(lr)
        xception_model.compile(optimizer = optimizer,
                              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                              metrics = ['accuracy'])
        
        return xception_model
    
    def fit(self, hp, model, X_train, y_train, validation_data = None, **kwargs):
        
        return model.fit(X_train, y_train,
                        validation_data = validation_data,
                        batch_size = hp.Choice('batch_size', [16,32,64]),
                        **kwargs,
                        )

In [20]:
## Tune with Bayesian Optimizer
tuner_bayesian = kt.BayesianOptimization(
        HyperModel(),
        objective = 'val_loss',
        max_trials = 50)

In [21]:
tuner_bayesian.search_space_summary()

Search space summary
Default search space size: 5
fc_dropout (Boolean)
{'default': False, 'conditions': []}
pooling_dropout (Boolean)
{'default': False, 'conditions': []}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
l2_conv2d (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.01], 'ordered': True}
l2_sepconv2d (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.01], 'ordered': True}


In [22]:
import os 
import numpy as np

In [23]:
import os

print(os.getcwd())
os.chdir('../datasets')
print(os.getcwd())

/home/jovyan/committed_git/datasets
/home/jovyan/committed_git/datasets


In [24]:
from sklearn.model_selection import train_test_split

# Import dataset
features = np.load('augmented_features_10_ue1_v2_ds.npy')
labels = np.load('augmented_labels_10_ue1_v2_ds.npy')

print(f'Shape of features np array: {features.shape}')
print(f'Shape of labels np array: {labels.shape}')

X = features
y = labels

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

Shape of features np array: (89628, 193, 16)
Shape of labels np array: (89628,)


In [25]:
print(os.getcwd())
os.chdir('../xception')
print(os.getcwd())

/home/jovyan/committed_git/datasets
/home/jovyan/committed_git/xception


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
tuner_bayesian.search(X_train, y_train,
            validation_data = (X_test, y_test),
            epochs = 100,
            callbacks = [stop_early])

Trial 42 Complete [00h 26m 10s]
val_loss: 0.2728256583213806

Best val_loss So Far: 0.19670678675174713
Total elapsed time: 14h 12m 52s

Search: Running Trial #43

Value             |Best Value So Far |Hyperparameter
False             |True              |fc_dropout
False             |True              |pooling_dropout
0.01              |0.0001            |learning_rate
0                 |0                 |l2_conv2d
0                 |0                 |l2_sepconv2d
32                |64                |batch_size

Epoch 1/100
2241/2241 [==============================] - 49s 20ms/step - loss: 7.5242 - accuracy: 0.0027 - val_loss: 11.3497 - val_accuracy: 0.0023
Epoch 2/100
2241/2241 [==============================] - 44s 20ms/step - loss: 5.5221 - accuracy: 0.0316 - val_loss: 6.2480 - val_accuracy: 0.0291
Epoch 3/100
2241/2241 [==============================] - 44s 20ms/step - loss: 3.9161 - accuracy: 0.1248 - val_loss: 3.9973 - val_accuracy: 0.1374
Epoch 4/100
2241/2241 [==============

In [ ]:
tuner_bayesian.results_summary()

In [ ]:
best_hp = tuner_bayesian.get_best_hyperparameters()[0]

In [ ]:
print(best_hp.values)

## Analysis of Results

In [17]:
print(os.getcwd()) 

/home/jovyan/committed_git/xception


In [19]:
# Analysis of results
import os
import json

print(os.getcwd())

for _, _, file_names in os.walk('tuning_results'):
    print(file_names)
    files = file_names    

tuning_results = {}

for file in files:
    cur_filename = 'tuning_results/' + file
    data = open(cur_filename)
    data = json.load(data)
    
    trial_results = {}
    trial_results['trial_id'] = data['trial_id']
    trial_results['values'] = data['hyperparameters']['values']
    trial_results['val_loss'] = data['metrics']['metrics']['val_loss']['observations'][0]['value'][0]
    trial_results['status'] = data['status']
    tuning_results[data['trial_id']] = trial_results

/home/jovyan/committed_git/xception
['trial09.json', 'trial06.json', 'trial04.json', 'trial08.json', 'trial10.json', 'trial01.json', 'trial05.json', 'trial00.json', 'trial07.json', 'trial03.json', 'trial02.json']


In [20]:
best_trial = min(tuning_results.keys(), key=lambda x: tuning_results[x]['val_loss'])

print(f'Trial that resulted in minimum validation loss: {best_trial}')
print(f'Validation Loss: {tuning_results[best_trial]["val_loss"]}')
print(f'Best Hyperparameters: {tuning_results[best_trial]["values"]}')

Trial that resulted in minimum validation loss: 00
Validation Loss: 0.25114965438842773
Best Hyperparameters: {'fc_dropout': True, 'pooling_dropout': False, 'learning_rate': 0.0001, 'l2_conv2d': 0.0, 'l2_sepconv2d': 0.01, 'batch_size': 16}


## Best Hyperparameters:
- fc_dropout: True
- pooling_dropout: False
- learning_rate: 0.0001
- l2_conv2d: 0.0
- l2_sepconv2d: 0.01
- batch_size: 16